## First inspection on raw data

In [12]:
import os
import pandas as pd


df = pd.read_csv("../../data/sample_temperature_data_for_coding_challenge.csv")
df.head()

,source_id,datetime,property_name,temperature
0,MICDEV001,2019-04-13T17:51:16.000+0000,heating_temperature,33.3
1,MICDEV001,2019-04-13T17:51:16.000+0000,cooling_temperature,15.0
2,MICDEV001,2019-04-13T18:51:18.000+0000,heating_temperature,34.0
3,MICDEV001,2019-04-13T19:51:20.000+0000,heating_temperature,33.8
4,MICDEV001,2019-04-13T20:51:21.000+0000,heating_temperature,34.2


In [13]:
df.dtypes

source_id         object
datetime          object
property_name     object
temperature      float64
dtype: object

Combine datapoints with identical timestamp

In [44]:
import csv
import numpy as np

sample_file = "../../data/sample_temperature_data_for_coding_challenge.csv"

with open(sample_file, "r") as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    new_table = []
    temp_line = None
    # iterate through rows and compare timestamp of last row (temp_line)
    # with time stamp of current row and join if equal
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
            line_count += 1
        else:
            if temp_line is None:
                if row[2] == "heating_temperature":
                    temp_line = [row[1],row[3],np.nan]
                else:
                    temp_line = [row[1],np.nan,row[3]]
            else:
                if temp_line[0] == row[1]:
                    if row[2] == "heating_temperature":
                        temp_line[1] = row[3]
                    else:
                        temp_line[2] = row[3]
                    new_table.append(temp_line)
                    temp_line = None
                else:
                    new_table.append(temp_line)
                    if row[2] == "heating_temperature":
                        temp_line = [row[1],row[3],np.nan]
                    else:
                        temp_line = [row[1],np.nan,row[3]]
            line_count += 1
    print(f'Processed {line_count} lines.')
    

Column names are source_id, datetime, property_name, temperature
Processed 1001 lines.


In [46]:
len(new_table)

716

In [47]:
df = pd.DataFrame(new_table, columns =['datetime', 'heating_temperature','cooling_temperature'])
df["datetime"] = pd.to_datetime(df["datetime"], format="%Y-%m-%dT%H:%M:%S.000+0000")
df["heating_temperature"] = pd.to_numeric(df["heating_temperature"])
df["cooling_temperature"] = pd.to_numeric(df["cooling_temperature"])
df.head()

,datetime,heating_temperature,cooling_temperature
0,2019-04-13 17:51:16,33.3,15.0
1,2019-04-13 18:51:18,34.0,NaN
2,2019-04-13 19:51:20,33.8,NaN
3,2019-04-13 20:51:21,34.2,NaN
4,2019-04-13 21:51:23,34.5,NaN
